In [119]:
import collections
import json
import numpy as np

In [163]:
location = collections.defaultdict()

with open('./Tucson/Tucson_location.json', mode="r") as f:

    line = f.readline()
    
    while line:
        
        line = json.loads(line)
        
        location[line['poi_id']] = (line['latitude'], line['longitude'])
        
        line = f.readline()

f.close()

np.save("./Tucson/location.npy", location, allow_pickle=True)

XXX = np.load("./Tucson/location.npy", allow_pickle=True).item()

print(type(XXX))

<class 'collections.defaultdict'>


In [121]:
# user_dict = collections.defaultdict(set)
# poi_dict = collections.defaultdict(set)

# with open('./Tucson/Tucson.json', mode="r") as f:

#   line = f.readline()

#   while line:

#     line = json.loads(line)
    
#     user_dict[line['user_id']].add(line['business_id'])
#     poi_dict[line['business_id']].add(line['user_id'])

#     line = f.readline()

# f.close()

In [122]:
# print(len(user_dict))
# print(len(poi_dict))

103342
8809


In [123]:
# delete_user = []
# delete_poi = []

# for k,v in user_dict.items():
#   if len(v) < 15:
#     delete_user.append(k)

# for k,v in poi_dict.items():
#   if len(v) < 10:
#     delete_poi.append(k)

In [124]:
# print(len(user_dict) - len(delete_user))
# print(len(poi_dict) - len(delete_poi))

2419
5147


In [125]:
# user_list = list(user_dict.keys())
# user_map = list(set(user_list).difference(set(delete_user)))
# print(len(user_map))

# poi_list = list(poi_dict.keys())
# poi_map = list(set(poi_list).difference(set(delete_poi)))
# print(len(poi_map))

2419
5147


In [126]:
# with open('./Tucson/Tucson.json', mode="r") as ff:
#     with open('./Tucson/Tucson_check.json', mode="w") as f:
        
#         line = ff.readline()
        
#         while line:
            
#             line = json.loads(line)
            
#             if line['user_id'] in user_map and line['business_id'] in poi_map:
            
# #                 line['user_name'] = line['user_id']
# #                 line['poi_name'] = line['business_id']
# #                 line['user_id'] = user_map.index(line['user_id'])
# #                 line['poi_id'] = poi_map.index(line['business_id'])
                
#                 line = json.dumps(line)
#                 f.write(line + "\n")
                
            
#             line = ff.readline()

#     f.close()
# ff.close()

In [127]:
# All = []

# xxx = set()
# yyy = set()

# with open('./Tucson/Tucson_check.json', mode="r") as f:

#   line = f.readline()

#   while line:

#     line = json.loads(line)
    
#     xxx.add(line['user_id'])
#     yyy.add(line['business_id'])
    
#     All.append(line)

#     line = f.readline()

# f.close()

# All = sorted(All, key=lambda x:x['date'])

In [128]:
# print(len(xxx))
# print(len(yyy))
# print(len(All))

2419
4964
86870


In [129]:
# xxx = list(xxx)
# yyy = list(yyy)

# with open('./Tucson/Tucson_check.json', mode="w") as f:

#   for sample in All:

#     temp = dict()
    
#     temp['user_name'] = sample['user_id']
#     temp['user_id'] = xxx.index(sample['user_id'])
#     temp['poi_name'] = sample['business_id']
#     temp['poi_id'] = yyy.index(sample['business_id'])
#     temp['date'] = sample['date']
    
#     temp = json.dumps(temp)
#     f.write(temp + "\n")

# f.close()

In [164]:
import time

DataSet = collections.defaultdict(list)

Map = collections.defaultdict()

with open('./Tucson/Tucson_check.json', mode='r') as f:
    with open("./Tucson/timestamp.txt", mode="w") as ff:
    
      line = f.readline()
    
      while line:

        sample = json.loads(line)
        
        date = sample['date']
        timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
        timestamp = int(time.mktime(timeArray))

        ff.write(str(sample['user_id']) + " " + str(sample['poi_id']) + " " +  str(timestamp) + "\n")

        DataSet[sample['user_id']].append(sample['poi_id'])

        Map[sample['user_name']] = sample['user_id']
#         Map[sample['poi_name']] = sample['poi_id']
        
        line = f.readline()
        
    ff.close()
f.close()

In [159]:
Category = list()

with open('./yelp_academic_dataset_business.json', mode="rb") as f:
    
    line = f.readline()
    
    while line:
        
        sample = json.loads(line)
        poi_id = sample['business_id']
        if poi_id in Map.keys():
            Category += sample['categories'].split(", ")

        line = f.readline()

f.close()

Category = set(Category)
category_Map = collections.Counter()

for k,v in enumerate(Category):
    category_Map[v] = k

    
with open('./yelp_academic_dataset_business.json', mode="rb") as f:
    with open("./Tucson/category.txt", mode="w") as ff:
        
        line = f.readline()
        
        while line:

            sample = json.loads(line)
            poi_id = sample['business_id']
            if poi_id in Map.keys():
                temp = str(Map[poi_id]) + " "
                category = sample['categories'].split(", ")
                for i in category:
                    temp += str(category_Map[i]) + "\n"
                    ff.write(temp)
                    temp = str(Map[poi_id]) + " "
            
            line = f.readline()
    
    ff.close()
f.close()

In [160]:
with open('./yelp_academic_dataset_business.json', mode="rb") as f:
  with open("./Tucson/Tucson_location.json", mode="w") as ff:

    line = f.readline()

    while line:

        sample = json.loads(line)

        temp = dict()
        poi_id = sample['business_id']
        if poi_id in Map.keys():
            
            temp['poi_id'] = Map[sample['business_id']]
            temp["latitude"] = float(sample["latitude"])
            temp["longitude"] = float(sample["longitude"])
            
            temp = json.dumps(temp)
            ff.write(temp + "\n")

        line = f.readline()

  ff.close()
f.close()

In [162]:
with open('./yelp_academic_dataset_user.json', mode="rb") as f:
  with open("./Tucson/social.txt", mode="w") as ff:

    line = f.readline()

    while line:
        
        sample = json.loads(line)
    
        user_id = sample['user_id']
        
        user_friends = sample['friends'].split(", ")
        
        if user_id in Map.keys():
            
            temp = str(Map[user_id]) + " "
            for friend in user_friends:
                if friend in Map.keys():
                    
                    temp += str(Map[friend]) + "\n"
#                     print(temp)
                    ff.write(temp)
                    temp = str(Map[user_id]) + " "
        
        line = f.readline()

  ff.close()
f.close()

In [165]:
import numpy as np

# for k,v in DataSet.items():

#     print(k,v)
#     break

# for k,v in DataSet.items():
    
#     DataSet[k] = collections.Counter(v)

# for k,v in DataSet.items():

#     temp = []
#     for p,n in v.items():
#         temp.append((p,n))
#     DataSet[k] = temp

# for k,v in DataSet.items():
#     print(k,v)
#     break

np.save("./Tucson/user_rating_set_all.npy", DataSet, allow_pickle=True)

In [148]:
trainSet = collections.defaultdict(list)
testSet = collections.defaultdict(list)
validSet = collections.defaultdict(list)


for k,v in DataSet.items():
  
  l = len(v)

  train = int(round(l*0.7))
  test = int(round(l*0.1))
  valid = int(round(l*0.2))

  l_ = l - (train + test + valid)

  if l_ == -1:
    train = train - 1
  if l_ == 1:
    test = test + 1
  
  trainSet[k] = v[0:train]
  testSet[k] = v[train:train+test]
  validSet[k] = v[train+test:]

for k,v in DataSet.items():

  x = list(set(trainSet[k]) & set(testSet[k]))
  y = list(set(trainSet[k]) & set(validSet[k]))

  trainSet[k] = list(set(trainSet[k]).difference(set(x)).difference(set(y)))
  testSet[k] = list(set(testSet[k]).difference(set(x)))
  validSet[k] = list(set(validSet[k]).difference(set(y)))

In [149]:
print(trainSet[99])
print(testSet[99])
print(validSet[99])

[(3400, 1), (2719, 1), (2752, 1), (3333, 1), (4233, 1), (3389, 1), (3696, 1), (3191, 1), (2108, 1), (1418, 1), (139, 1), (1225, 1), (818, 1), (2690, 1), (3086, 1), (3472, 1), (2484, 1), (4852, 1), (2039, 1), (2336, 1), (78, 1), (2248, 1)]
[(4860, 1), (2822, 1), (2204, 1)]
[(2485, 1), (3933, 1), (4564, 1), (3386, 1), (2423, 1), (855, 1)]


In [137]:
# np.save("./Tucson/testList.npy", testSet, allow_pickle=True)

# XXX = np.load("./Tucson/testList.npy", allow_pickle=True).item()

# print(type(XXX))

<class 'collections.defaultdict'>


In [157]:
with open('./Tucson/new_test.txt', mode="w") as f:

    for k,v in testSet.items():
        temp = str(k) + " "
        for i in v:
            temp += str(i[0]) + " " + str(i[1]) + "\n"
            f.write(temp)
            temp = str(k) + " "

f.close()